### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [2]:
import pandas as pd
df = pd.read_csv("food_transactions.csv", index_col = 0)
df

,date,branch,order
0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink"
1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink"
2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink"
3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink"
4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink"
...,...,...,...
134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink"
134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries"
134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries"
134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink"


In [3]:
# 1. Given the order column, count how many of each product was purchased

def burger_counter(order):
    if type(order) == str and "S_Burger" in order:
        return int(order[order.find("S_Burger") - 1])
    else:
        return 0

def fries_counter(order):
    if type(order) == str and "S_Fries" in order:
        return int(order[order.find("S_Fries") - 1])
    else:
        return 0

def drink_counter(order):
    if type(order) == str and "S_Drink" in order:
        return int(order[order.find("S_Drink") - 1])
    else:
        return 0

df["No. of Burgers"] = df["order"].apply(burger_counter)
df["No. of Fries"] = df["order"].apply(fries_counter)
df["No. of Drinks"] = df["order"].apply(drink_counter)
df

,date,branch,order,No. of Burgers,No. of Fries,No. of Drinks
0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5
1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3
2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2
3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,6,1
4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6
...,...,...,...,...,...,...
134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,3,6
134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0
134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0
134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5


In [4]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks

df["Fam_Combo_Burger"] = df["No. of Burgers"] // 4
df["Fam_Combo_Fries"] = df["No. of Fries"] // 4
df["Fam_Combo_Drink"] = df["No. of Drinks"] // 4
df["Family_Combo"] = df[["Fam_Combo_Burger", "Fam_Combo_Fries", "Fam_Combo_Drink"]].min(axis = 1)

df = df.drop("Fam_Combo_Burger", axis = 1)
df = df.drop("Fam_Combo_Fries", axis = 1)
df = df.drop("Fam_Combo_Drink", axis = 1)

df["placeholder_burger"] = df["No. of Burgers"] - 4 * df["Family_Combo"]
df["placeholder_fries"] = df["No. of Fries"] - 4 * df["Family_Combo"]
df["placeholder_drinks"] = df["No. of Drinks"] - 4 * df["Family_Combo"]
df["Big_Combo"] = df[["placeholder_burger", "placeholder_fries", "placeholder_drinks"]].min(axis = 1)

df = df.drop("placeholder_burger", axis = 1)
df = df.drop("placeholder_fries", axis = 1)
df = df.drop("placeholder_drinks", axis = 1)

df["placeholder_fries"] = df["No. of Fries"] - 4 * df["Family_Combo"] - df["Big_Combo"]
df["placeholder_drinks"] = df["No. of Drinks"] - 4 * df["Family_Combo"] - df["Big_Combo"]
df["Snack_Combo"] = df[["placeholder_fries", "placeholder_drinks"]].min(axis = 1)

df = df.drop("placeholder_fries", axis = 1)
df = df.drop("placeholder_drinks", axis = 1)

df["S_Burger"] = df["No. of Burgers"] - 4 * df["Family_Combo"] - df["Big_Combo"]
df["S_Fries"] = df["No. of Fries"] - 4 * df["Family_Combo"] - df["Big_Combo"] - df["Snack_Combo"]
df["S_Drink"] = df["No. of Drinks"] - 4 * df["Family_Combo"] - df["Big_Combo"] - df["Snack_Combo"]

df

,date,branch,order,No. of Burgers,No. of Fries,No. of Drinks,Family_Combo,Big_Combo,Snack_Combo,S_Burger,S_Fries,S_Drink
0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5,0,1,3,0,0,1
1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3,0,2,1,0,1,0
2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2,0,2,0,0,2,0
3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,6,1,0,0,1,0,5,0
4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6,0,0,0,5,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,3,6,0,0,3,0,0,3
134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0,0,0,0,7,6,0
134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0,0,0,0,4,6,0
134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5,1,1,0,2,1,0


In [8]:
# 3. Calculate the total order value.

jsondf = pd.read_json("price_list.json")

df["Revenue"] = df["Family_Combo"] * jsondf["Fam_combo"] + df["Big_Combo"] * jsondf["Big_combo"] + df["Snack_Combo"] * jsondf["snack_combo"] + df["S_Burger"] * jsondf["S_Burger"] + df["S_Fries"] * jsondf["S_Fries"] + df["S_Drink"] * jsondf["S_Drink"]


KeyError: 'Fam_combo'

In [ ]:
# 4. Determine which branch made the most revenue, and how much?

df.groupby(["branch"]).sum(numeric_only = True)["Revenue"].sort_values(asecnding = False)